In [29]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc, classification_report, roc_curve
import warnings
warnings.filterwarnings('ignore')

In [31]:
df=pd.read_csv("D:/cdrive/desktop/DIVYA/CAREER/guvi/PROJECT/CREDIT_CARD/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
'''
This data is about the credit card fraud transactions.
Class column having 0 indicates a normal transaction.
Class column having 1 indicates a fraud transaction.
'''

'\nThis data is about the credit card fraud transactions.\nClass column having 0 indicates a normal transaction.\nClass column having 1 indicates a fraud transaction.\n'

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [35]:
# Getting the shape of the data
print('Shape :', df.shape)
print('Rows :', df.shape[0])
print('Columns :', df.shape[1])

Shape : (284807, 31)
Rows : 284807
Columns : 31


In [37]:
# Getting an idea about duplicate data
df.duplicated().sum()

1081

In [39]:
# Removing the duplicate rows
df.drop_duplicates(inplace = True)

In [41]:
# Getting an idea about duplicate data
df.duplicated().sum()

0

In [43]:
# Getting an idea about null values
df.isnull().sum().any()

False

In [45]:
# Normalizing the amount column
df['normAmount'] = StandardScaler().fit_transform(np.array(df['Amount']).reshape(-1, 1))

# Dropping the columns
df.drop(['Time', 'Amount'], inplace = True, axis = 1)

In [47]:
# Getting the proportion of values from 'Class' column
df['Class'].value_counts(normalize = True).round(4)

Class
0    0.9983
1    0.0017
Name: proportion, dtype: float64

In [12]:
# We have 99.83% of the data from Class 0
# We have 0.17% of the data from Class 1

In [13]:
# The above data indicates that there is a huge amount of imbaalance in the data

In [49]:
# Seperating input features and target variable
X = df.drop('Class', axis = 1)
y = df['Class']

# Performing train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)

In [51]:
# Creating the model
xgb_clf = XGBClassifier(random_state=5, use_label_encoder=False, eval_metric='logloss')

# Fitting the model
xgb_clf.fit(X_train, y_train)

# Predicting from the model (Testing Data)
y_pred = xgb_clf.predict(X_test)

# Predicting from the model (Training Data)
y_train_pred = xgb_clf.predict(X_train)

In [53]:
# Model Evaluation
# Testing Dataset
print('Testing Accuracy :', np.round(metrics.accuracy_score(y_test, y_pred), 2))
print('Testing Precision :', np.round(metrics.precision_score(y_test, y_pred), 2))
print('Testing Recall :', np.round(metrics.recall_score(y_test, y_pred), 2))
print('Testing F1 Score :', np.round(metrics.f1_score(y_test, y_pred), 2))

# Training Dataset
print('\nTraining Accuracy :', np.round(metrics.accuracy_score(y_train, y_train_pred), 2))
print('Training Precision :', np.round(metrics.precision_score(y_train, y_train_pred), 2))
print('Training Recall :', np.round(metrics.recall_score(y_train, y_train_pred), 2))
print('Training F1 Score :', np.round(metrics.f1_score(y_train, y_train_pred), 2))
print("\n Classification Report:\n", classification_report(y_test, y_pred))

Testing Accuracy : 1.0
Testing Precision : 0.97
Testing Recall : 0.84
Testing F1 Score : 0.9

Training Accuracy : 1.0
Training Precision : 1.0
Training Recall : 1.0
Training F1 Score : 1.0

 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56654
           1       0.97      0.84      0.90        92

    accuracy                           1.00     56746
   macro avg       0.99      0.92      0.95     56746
weighted avg       1.00      1.00      1.00     56746



In [ ]:
'''
XGBoost model achieves 100% accuracy on the test set, indicating potential overfitting,
as the training accuracy is also perfect. 
While precision (97%) for fraud detection is high (few false alarms), recall (84%) shows that 16% of fraud cases are missed,
which could be critical in real-world scenarios. 
The F1-score (0.90) suggests a good balance, but improving recall is crucial.
The model performs exceptionally well for legitimate transactions (class 0), 
but class imbalance still affects fraud detection.
'''

In [57]:
import time
import numpy as np
from scipy.stats import randint, uniform  
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE  # Import SMOTE

# Load or Prepare Dataset
# Assuming X and y are already preprocessed feature matrix and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Debugging: Check dataset sizes
print("X_train_res shape:", X_train_res.shape)
print("y_train_res shape:", y_train_res.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Define Hyperparameter Search Space
param_dist = {
    'n_estimators': randint(50, 200),          
    'learning_rate': uniform(0.01, 0.3),       
    'max_depth': randint(3, 10),               
    'min_child_weight': randint(1, 10),        
    'subsample': uniform(0.6, 0.4),            
    'colsample_bytree': uniform(0.6, 0.4),     
    'gamma': uniform(0, 0.5),                  
    'reg_lambda': uniform(0.1, 1.0)            
}

# Initialize XGBoost Classifier
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# RandomizedSearchCV for Hyperparameter Tuning
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,  
    cv=3,  # 3-fold cross-validation
    scoring='f1',  
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Start Timer
start_time = time.time()

# Fit the model with resampled training data
random_search.fit(X_train_res, y_train_res)

# End Timer
end_time = time.time()

# Best Hyperparameters
best_params = random_search.best_params_
print("\nBest Hyperparameters:", best_params)
print(f"Hyperparameter Tuning Time: {end_time - start_time:.2f} seconds\n")

# Train Final Model with Best Parameters
best_xgb = XGBClassifier(**best_params, random_state=42, use_label_encoder=False, eval_metric='logloss')
best_xgb.fit(X_train_res, y_train_res)  # Use SMOTE data

# Predictions
y_train_pred = best_xgb.predict(X_train_res)
y_test_pred = best_xgb.predict(X_test)

# Model Evaluation - Training Data
print("\nTraining Set Evaluation Metrics:")
print(f"Accuracy  : {accuracy_score(y_train_res, y_train_pred):.4f}")
print(f"Precision : {precision_score(y_train_res, y_train_pred, average='weighted'):.4f}")
print(f"Recall    : {recall_score(y_train_res, y_train_pred, average='weighted'):.4f}")
print(f"F1 Score  : {f1_score(y_train_res, y_train_pred, average='weighted'):.4f}")

# Model Evaluation - Testing Data
print("\nTesting Set Evaluation Metrics:")
print(f"Accuracy  : {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Precision : {precision_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"Recall    : {recall_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"F1 Score  : {f1_score(y_test, y_test_pred, average='weighted'):.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


X_train_res shape: (453204, 29)
y_train_res shape: (453204,)
X_test shape: (56746, 29)
y_test shape: (56746,)
Fitting 3 folds for each of 10 candidates, totalling 30 fits

Best Hyperparameters: {'colsample_bytree': 0.6557975442608167, 'gamma': 0.14607232426760908, 'learning_rate': 0.1199085529881075, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 157, 'reg_lambda': 0.6142344384136116, 'subsample': 0.836965827544817}
Hyperparameter Tuning Time: 137.16 seconds


Training Set Evaluation Metrics:
Accuracy  : 1.0000
Precision : 1.0000
Recall    : 1.0000
F1 Score  : 1.0000

Testing Set Evaluation Metrics:
Accuracy  : 0.9992
Precision : 0.9992
Recall    : 0.9992
F1 Score  : 0.9992

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56651
           1       0.74      0.79      0.77        95

    accuracy                           1.00     56746
   macro avg       0.87      0.89      0.88     56746
weighted avg

In [ ]:
'''
XGBoost model achieves 99.92% accuracy, but this is mainly due to the dominance of legitimate transactions.
The training metrics (100%) indicate overfitting, suggesting the model has memorized the data.
Fraud detection (Class 1) shows 74% precision and 79% recall, meaning some fraud cases are missed,
and false positives exist. SMOTE improved recall, but the model still leans toward the majority class..
'''

In [61]:
from imblearn.over_sampling import ADASYN   

# Load or Prepare Dataset
# Assuming X and y are already preprocessed feature matrix and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply ADASYN to handle class imbalance
adasyn = ADASYN(random_state=42)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)

# Debugging: Check dataset sizes
print("X_train_res shape:", X_train_res.shape)
print("y_train_res shape:", y_train_res.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Define Hyperparameter Search Space
param_dist = {
    'n_estimators': randint(50, 200),          
    'learning_rate': uniform(0.01, 0.3),       
    'max_depth': randint(3, 10),               
    'min_child_weight': randint(1, 10),        
    'subsample': uniform(0.6, 0.4),            
    'colsample_bytree': uniform(0.6, 0.4),     
    'gamma': uniform(0, 0.5),                  
    'reg_lambda': uniform(0.1, 1.0)            
}

# Initialize XGBoost Classifier
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# RandomizedSearchCV for Hyperparameter Tuning
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,  
    cv=3,  # 3-fold cross-validation
    scoring='f1',  
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Start Timer
start_time = time.time()

# Fit the model with resampled training data
random_search.fit(X_train_res, y_train_res)

# End Timer
end_time = time.time()

# Best Hyperparameters
best_params = random_search.best_params_
print("\nBest Hyperparameters:", best_params)
print(f"Hyperparameter Tuning Time: {end_time - start_time:.2f} seconds\n")

# Train Final Model with Best Parameters
best_xgb = XGBClassifier(**best_params, random_state=42, use_label_encoder=False, eval_metric='logloss')
best_xgb.fit(X_train_res, y_train_res)  # Use SMOTE data

# Predictions
y_train_pred = best_xgb.predict(X_train_res)
y_test_pred = best_xgb.predict(X_test)

# Model Evaluation - Training Data
print("\nTraining Set Evaluation Metrics:")
print(f"Accuracy  : {accuracy_score(y_train_res, y_train_pred):.4f}")
print(f"Precision : {precision_score(y_train_res, y_train_pred, average='weighted'):.4f}")
print(f"Recall    : {recall_score(y_train_res, y_train_pred, average='weighted'):.4f}")
print(f"F1 Score  : {f1_score(y_train_res, y_train_pred, average='weighted'):.4f}")

# Model Evaluation - Testing Data
print("\nTesting Set Evaluation Metrics:")
print(f"Accuracy  : {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Precision : {precision_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"Recall    : {recall_score(y_test, y_test_pred, average='weighted'):.4f}")
print(f"F1 Score  : {f1_score(y_test, y_test_pred, average='weighted'):.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


X_train_res shape: (453241, 29)
y_train_res shape: (453241,)
X_test shape: (56746, 29)
y_test shape: (56746,)
Fitting 3 folds for each of 10 candidates, totalling 30 fits

Best Hyperparameters: {'colsample_bytree': 0.6849356442713105, 'gamma': 0.09091248360355031, 'learning_rate': 0.06502135295603015, 'max_depth': 6, 'min_child_weight': 6, 'n_estimators': 138, 'reg_lambda': 0.3912291401980419, 'subsample': 0.8447411578889518}
Hyperparameter Tuning Time: 108.72 seconds


Training Set Evaluation Metrics:
Accuracy  : 0.9979
Precision : 0.9979
Recall    : 0.9979
F1 Score  : 0.9979

Testing Set Evaluation Metrics:
Accuracy  : 0.9952
Precision : 0.9984
Recall    : 0.9952
F1 Score  : 0.9965

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56651
           1       0.24      0.83      0.37        95

    accuracy                           1.00     56746
   macro avg       0.62      0.91      0.68     56746
weighted a

In [ ]:
'''
The model achieves 99.52% accuracy, correctly classifying most transactions, but struggles with fraud detection.
High recall (83%) for fraud cases means it detects most frauds, but low precision (24%) indicates many false positives. 
Legitimate transactions (Class 0) are almost perfectly classified, but fraud detection needs improvement. 
'''